In [1]:
#import necessary libraries
import os
import copy
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
from torchvision import models
from sklearn.utils import shuffle
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.font_manager
from collections import OrderedDict

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
import os
print(os.getcwd())


c:\Users\Renuka\Desktop\prodigy task 5


 **understanding the dataset**

In [43]:
if "food-101" in os.listdir():
    print("Dataset already exists")
else:
    print("Downloading the data...")
    !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
    print("Dataset downloaded!")
    print("Extracting data..")
    !tar xzvf food-101.tar.gz > /dev/null 2>&1
    print("Extraction done!")

Dataset downloaded!
Extracting data..


'wget' is not recognized as an internal or external command,
operable program or batch file.


Extraction done!


The system cannot find the path specified.


In [5]:
import os

file_path = "./food-101/meta/classes.txt"

if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        classes = file.read().splitlines()
    classes_21 = classes[:20] + ['other']
    print(classes_21, len(classes_21))
else:
    print(f"File not found: {file_path}")


File not found: ./food-101/meta/classes.txt


**identifying training and testing images**

In [44]:
#The code snippet displays the first five entries from two files, test.txt and train.txt, located in the food-101/meta/ directory. It begins by printing "Testing images," followed by the first five lines from test.txt, then prints "Training images" and shows the first five lines from train.txt. The purpose is to preview the contents of these files, likely listing image data for testing and training purposes.
!echo "Testing images"
!head -n 5 ./food-101/meta/test.txt
!echo -e "\nTraining images"
!head -n 5 ./food-101/meta/train.txt | head -n 5

"Testing images"


'head' is not recognized as an internal or external command,
operable program or batch file.


-e "\nTraining images"


'head' is not recognized as an internal or external command,
operable program or batch file.


**exploratory data analysis (for dummies)**

In [7]:
import pandas as pd

def prep_df(path: str) -> pd.DataFrame:
    array = open(path, 'r').read().splitlines()
    
    # Getting the full path for the images
    df = pd.DataFrame(array, columns=['relative_path'])
    
    # Shuffling the rows of the DataFrame randomly
    df = df.sample(frac=1).reset_index(drop=True)
    
    return df


In [8]:
import pandas as pd

def prep_df(path: str) -> pd.DataFrame:
    array = open(path, 'r').read().splitlines()
    
    # Getting the full path for the images
    df = pd.DataFrame(array, columns=['relative_path'])
    
    # Shuffling the rows of the DataFrame randomly
    df = df.sample(frac=1).reset_index(drop=True)
    
    return df


In [9]:
import matplotlib.pyplot as plt

# Now you can create your figure
plt.figure(figsize=(20, 5))

num_rows = 3
num_cols = 8

# Your code for visualizing the images goes here


<Figure size 2000x500 with 0 Axes>

**visualization using barplot**

In [10]:
def barplot_vis(imgs_dataframe):# Use the newly integrated Roboto font family for all text.
    fig, ax = plt.subplots()

    new_labels = [row if row in classes_21 else "other" for row in imgs_dataframe.label]
    tmp_imgs_dataframe = imgs_dataframe.copy(deep=True)
    tmp_imgs_dataframe['label'] = new_labels

    grouped_train_imgs = tmp_imgs_dataframe.groupby("label")

    heights = [grouped_train_imgs.get_group(group).shape[0] for group in classes_21]

    # Save the chart so we can loop through the bars below.
    bars = ax.bar(
        x=classes_21,
        height=heights,
        tick_label=classes_21
    )

    # Axis formatting.
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_color('#DDDDDD')
    ax.tick_params(bottom=False, left=False)
    ax.set_axisbelow(True)
    ax.yaxis.grid(True, color='#EEEEEE')
    ax.xaxis.grid(False)

    # Add text annotations to the top of the bars.
    bar_color = bars[0].get_facecolor()
    percentage_heights = np.array(heights) / sum(heights)
    for idx in range(len(bars)):
      ax.text(
          bars[idx].get_x() + bars[idx].get_width() / 2,
          bars[idx].get_height() + 0.3,
          round(percentage_heights[idx] * 100, 2),
          horizontalalignment='center',
          color=bar_color,
          weight='bold'
      )

    # Add labels and a title.
    ax.set_xlabel('Food Class', labelpad=15, color='#333333')
    ax.set_ylabel('Number of Images', labelpad=15, color='#333333')
    ax.set_title('Visualizing Class Imbalance', pad=15, color='#333333',
                 weight='bold')

    fig.autofmt_xdate(rotation=45)
    fig.set_size_inches(18.5, 4)

In [11]:
import os

file_path = "./food-101/meta/train.txt"

if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        array = file.read().splitlines()
    df = pd.DataFrame(array, columns=['relative_path'])
    df = df.sample(frac=1).reset_index(drop=True)
    print(df.head())
else:
    print(f"File not found: {file_path}")


File not found: ./food-101/meta/train.txt


**preparing the data for transformation**

In [12]:
# import torchvision.transforms as transforms

# Data augmentation for training
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Data augmentation for testing
test_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [13]:
## Assuming you have a file 'classes.txt' with the class names
file_path = "./food-101/meta/classes.txt"

if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        classes = file.read().splitlines()
else:
    print(f"File not found: {file_path}")


File not found: ./food-101/meta/classes.txt


In [14]:
from torch.utils.data import Dataset

class Food20(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        image = Image.open(img_path)
        label = self.dataframe.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label


In [15]:
#from torch.utils.data import Dataset
from PIL import Image

class Food20(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        image = Image.open(img_path)
        label = self.dataframe.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label


In [42]:
import os
import shutil

# Example paths
source_dir = 'C:/Users/Renuka/Documents/all_images'
target_dir = 'C:/Users/Renuka/Documents/train_data'

# Example class names and corresponding image prefixes
classes = ['class1', 'class2', 'class3']
class_prefixes = {'class1': 'prefix1_', 'class2': 'prefix2_', 'class3': 'prefix3_'}

# Create class directories
for class_name in classes:
    class_dir = os.path.join(target_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

# Verify and create the source directory if it doesn't exist
if not os.path.exists(source_dir):
    os.makedirs(source_dir)

# Move images to corresponding class directories
if os.path.exists(source_dir):
    for filename in os.listdir(source_dir):
        for class_name, prefix in class_prefixes.items():
            if filename.startswith(prefix):
                shutil.move(os.path.join(source_dir, filename), os.path.join(target_dir, class_name, filename))
                break
else:
    print(f"The directory {source_dir} does not exist.")


In [17]:
# import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms


**creating the ML model**

In [18]:
import torchvision.models as models

# Fetch the pretrained weights for DenseNet-201 trained on ImageNet
weights = models.DenseNet201_Weights.IMAGENET1K_V1

# Initialize the DenseNet-201 model using these weights
model = models.densenet201(weights=weights)


In [19]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt



In [40]:
import socket

try:
    # Attempt to resolve the hostname
    print(socket.getaddrinfo('github.com', 443))
except socket.gaierror as e:
    print(f"An error occurred: {e}")


[(<AddressFamily.AF_INET: 2>, 0, 0, '', ('20.207.73.82', 443))]


In [21]:
import torch.nn as nn

# Now you can define your classifier
classifier = nn.Sequential(
    nn.Linear(1920, 1024),
    nn.LeakyReLU(),
    nn.Linear(1024, 101),
)

# Assuming you have a pre-trained DenseNet-201 model
model = models.densenet201(pretrained=True)
model.classifier = classifier

# Load the checkpoint
checkpoint_path = "./food_classifier.pt"
model.load_state_dict(torch.load(checkpoint_path, map_location='cpu'), strict=False)


c:\Users\Renuka\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Renuka\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Renuka\AppData\Local\Temp\ipykernel_16180\1169004005.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during

<All keys matched successfully>

In [22]:
#hyper parameters
num_epochs = 3


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt


**training and testing the model**

In [25]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device):
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  print("--> Training Progress")
  # Loop through data loader data batches
  for batch, (X, y) in enumerate(tqdm(dataloader)):
      # Send data to target device
      images, labels = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(images)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, labels)
      train_loss += loss.item()

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == labels).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

In [26]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device):
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      print("--> Testing Progress")
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(tqdm(dataloader)):
          # Send data to target device
          images, labels = X.to(device), y.to(device)

          # 1. Forward pass
          test_pred_logits = model(images)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred_logits, labels)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          test_pred_labels = torch.argmax(torch.softmax(test_pred_logits, dim=1), dim=1)

          test_acc += ((test_pred_labels == labels).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

the code provides the machinery to train a deep learning model, assess its performance on a test dataset, and track and save the best-performing version of the model.

In [27]:
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device):
  # Create empty results dictionary
  history = {
      "train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": [],
      'best train acc': (0, 0),
      "best_model": dict()
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in range(epochs):
      print('Epoch {}/{}'.format(epoch + 1, num_epochs))

      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
          f"\n\n=============================\n"
      )

      # Update results dictionary
      history["train_loss"].append(train_loss)
      history["train_acc"].append(train_acc)
      history["test_loss"].append(test_loss)
      history["test_acc"].append(test_acc)
      if test_loss < history["test_acc"][len(history["test_acc"]) - 1]:
          history["best_model"] = model.state_dict()

      if test_acc > 0.95:
         break

  # Return the filled results at the end of the epochs
  return model, history

In [28]:
def train(model, train_loader, val_loader, optimizer, criterion, num_epochs, device):
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': [], 'best_model': None}
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc.item())

        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc.item())

        if val_acc > best_acc:
            best_acc = val_acc
            history['best_model'] = model.state_dict()

        print(f'Epoch {epoch}/{num_epochs - 1}')
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

    return model, history


**viewing the learning history**

In [29]:
#this function is a utility to help visualize and compare the performance of a model on training and validation datasets across epochs.
def plot_history(history):

    loss = history['train_loss']
    accuracy = history['train_acc']
    val_loss = history['test_loss']
    val_accuracy = history['test_acc']
    x = range(len(loss))

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, accuracy, label='Training acc', color='#03045e', linewidth=2)
    if len(val_loss) != 0:
      plt.plot(x, val_accuracy, label='Validation acc', color='#48cae4', linewidth=2)
    plt.plot(history['best train acc'][0],
             history['best train acc'][1],
             label='Best train acc', markersize=7, color='black')
    plt.title('Accuracy')
    plt.grid(True)
    plt.legend()


    plt.subplot(1, 2, 2)
    plt.plot(x, loss, label='Training loss', color='#03045e', linewidth=2)
    if len(val_loss) != 0:
      plt.plot(x, val_loss, label='Validation loss', color='#48cae4', linewidth=2)
    plt.title('Loss')
    plt.grid(True)
    plt.legend()

In [30]:
import matplotlib.pyplot as plt

def plot_history(history):
    plt.figure(figsize=(12, 4))

    # Plot training & validation loss values
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()


**evaluating the result using the required 21 classes only**

In [31]:
#the evaluate function measures and prints the classification accuracy of the model on a given dataset after adjusting classes to fit a 21-class problem (20 specific classes plus one "other" class).
def evaluate(model, dataloader):

  random = np.random.randint(0, len(dataloader))

  with torch.no_grad():
    model.eval()
    n_correct = 0
    n_samples = 0

    for images, labels in tqdm(dataloader):
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)

      preds = torch.argmax(torch.softmax(outputs, 1), 1)

      # Converting this problem to a problem with 21 clases only
      preds = np.array([pred.cpu() if pred < 20 else 20 for pred in preds])
      labels = np.array([label.cpu() if label < 20 else 20 for label in labels])

      n_samples += labels.shape[0]
      n_correct += (preds==labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(acc)

In [32]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.double() / len(dataloader.dataset)

    return epoch_loss, epoch_acc


In [33]:
# Assuming you have a file 'classes.txt' with the class names
file_path = "./food-101/meta/classes.txt"

if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        classes = file.read().splitlines()
    classes_21 = classes[:20] + ['other']
else:
    print(f"File not found: {file_path}")


File not found: ./food-101/meta/classes.txt


**visually evaluating the results**

In [34]:
#The visually_evaluate function aims to visually inspect the predictions made by a provided model using a random subset of images from a given DataFrame data_df.
def visually_evaluate(model, data_df):
    num_rows = 3
    num_cols = 6

    random_idx = np.random.randint(0, data_df.shape[0], num_rows*num_cols)

    # Retrieve a number of images from the dataset.
    imgs, labels = data_df.iloc[random_idx], data_df.iloc[random_idx].label
    eval_dataset = Food20(imgs, transform=test_transforms)
    eval_loader = DataLoader(eval_dataset, batch_size=num_rows*num_cols, shuffle=False)
    eval_set = next(iter(eval_loader))

    # Get predictions from model.
    model.eval()
    preds = model(eval_set[0].to(device))
    preds = torch.argmax(torch.softmax(preds, dim=1), dim=1)

    # Converting this problem to a 21 class problem
    preds = np.array([encoder_21.get_label(pred.cpu()) if pred < 20 else "other" for pred in preds])
    labels = np.array([label if encoder_21.get_idx(label) < 20 else "other" for label in labels])

    plt.figure(figsize=(20, 8))

    num_matches = 0
    for idx in range(num_rows*num_cols):
        ax = plt.subplot(num_rows, num_cols, idx + 1)
        plt.axis("off")

        img = plt.imread(imgs.path.iloc[idx])
        plt.imshow(img)

        title = labels[idx] + " : " + preds[idx]
        title_obj = plt.title(title, fontdict={'fontsize':13})

        if labels[idx] == preds[idx]:
            num_matches += 1
            plt.setp(title_obj, color='g')
        else:
            plt.setp(title_obj, color='r')

        acc = num_matches/(idx+1)
    print("Prediction accuracy: ", int(100*acc)/100)

    return

In [39]:
from torchvision import models
import torch.nn as nn

# Load a pre-trained DenseNet model
model = models.densenet121(pretrained=True)

# Get the number of input features for the classifier
num_ftrs = model.classifier.in_features

# Modify the classifier layer to match the number of classes in your dataset
model.classifier = nn.Linear(num_ftrs, len(classes))

# Move the model to the appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\Renuka/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:23<00:00, 1.39MB/s]


**saving the model results**

In [36]:
# Example of defining history with a best_model key
history = {
    'best_model': model.state_dict()  # Assuming model is your trained model
}

# Now you can save the best model's state dictionary
torch.save(history['best_model'], "./solution.pth")


In [37]:
import os

if os.path.exists("./solution.pth"):
    print("solution.pth exists in the current directory.")
else:
    print("solution.pth does not exist in the current directory.")


solution.pth exists in the current directory.
